# Cartpoleを試す

Stable BaselinesとRL Baselines Zooを用いて、強化学習によるバランス制御を試します。
このノートブックは以下の内容を含みます。

- 環境準備
- Gym環境とエージェントを作成
- エージェントの学習と評価
- 1行のコマンドで学習
- リプレイ動画の生成

なお、GIFアニメーションによる学習前後のプレイの可視化は、ikeyasu氏の[ChainerRL を Colaboratory で動かす - Qiita](https://qiita.com/ikeyasu/items/ec3c88ce13a2d5e41f26) を参考として作成しました。

## A. 環境を準備する

Stable Baselinesと依存ライブラリをインストールします。

### 1. 必要なライブラリのインストール

インストールに5分程度を要します。

In [ ]:
# added on 2020/2/9
%tensorflow_version 1.x
# エラー解消のため最新のstable-baselinesをインストール
!pip install optuna
!git clone https://github.com/hill-a/stable-baselines
%cd stable-baselines
!pip install -e .[docs,tests]

In [0]:
!apt-get -y install swig xvfb python-opengl
!pip install box2d box2d-kengz pybullet pyyaml pytablewriter pyvirtualdisplay

### 2. 必要なPythonライブラリのインポート


In [0]:
import time, os, gym
import numpy as np

from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines.common import set_global_seeds
from stable_baselines import DQN

from pyvirtualdisplay import Display

import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML

## B. Gym環境を準備する

CartPole-v1という、OpenAI Gymの中の、棒のバランスをとるタスク用の環境を準備します。

In [0]:
env_id = "CartPole-v1"
env = gym.make(env_id)
env = DummyVecEnv([lambda: env])

## C. その他の処理を準備する

Gym環境以外のヘルパー関数などを準備します。

### 1. 評価方法の準備

次の関数は、強化学習のエージェントを、環境内で指定のステップ分動かし、その結果得られる報酬を算出します。

In [0]:
def evaluate(model, num_steps=1000):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_steps: (int) number of timesteps to evaluate it
    :return: (float) Mean reward
    """
    episode_rewards = [[0.0] for _ in range(env.num_envs)]
    obs = env.reset()
    for i in range(num_steps):
      # _states are only useful when using LSTM policies
      actions, _states = model.predict(obs)
      # here, action, rewards and dones are arrays
      # because we are using vectorized env
      obs, rewards, dones, info = env.step(actions)
      
      # Stats
      for i in range(env.num_envs):
          episode_rewards[i][-1] += rewards[i]
          if dones[i]:
              episode_rewards[i].append(0.0)

    mean_rewards =  [0.0 for _ in range(env.num_envs)]
    n_episodes = 0
    for i in range(env.num_envs):
        mean_rewards[i] = np.mean(episode_rewards[i])     
        n_episodes += len(episode_rewards[i])   

    # Compute mean reward
    mean_reward = round(np.mean(mean_rewards), 1)
    print("Mean reward:", mean_reward, "Num episodes:", n_episodes)

    return mean_reward


### 2. プレイ動画の再生用関数

次に、仮想ディプレイを利用し、Colaboratory上でエージェントの振る舞いをアニメーションで見られるようにする関数を定義します。

In [0]:
def playback(model, env, maxsteps):
  # Start virtual display
  display = Display(visible=0, size=(1024, 768))
  display.start()

  # commented out on 2020/6/20
  # os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

  frames = []
  for i in range(3):
      obs = env.reset()
      done = False
      R = 0
      t = 0
      while not done and t < maxsteps:
          frames.append(env.render(mode = 'rgb_array'))
          action, _states = model.predict(obs)        
          obs, rewards, dones, info = env.step(action)
          R += rewards
          t += 1
      print('test episode:', i, 'R:', R)
  #    model.stop_episode()
  #env.render()

  return frames

## D. モデルを作成する


### 1. エージェントの準備

さきほど `env` に設定した `CartPole-v1` 環境を使って学習する、DQNのエージェントを作成します。


In [0]:
model = DQN(MlpPolicy, env, verbose=1, tensorboard_log="./cartpole_tensorboard/")

### 3. 学習前エージェントの評価

さきほど設定した評価関数を、学習前のまっさらなエージェントに対して適用してみましょう。

実行すると平均の報酬(Mean reward)と、エピソード数(Num episodes)が得られます。学習後の実行と比べてみましょう。

In [0]:
# Random Agent, before training
mean_reward_before_train = evaluate(model, num_steps=1000)

### 4. 未学習状態での振る舞いを見る

この未学習状態でのプレイぶりを再生してみましょう。

In [0]:
frames = playback(model, env, 50)

plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames[0])
plt.axis('off')
animate = lambda i: patch.set_data(frames[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)
HTML(ani.to_jshtml())

すぐに棒が倒れてしまい、プレイがあまり継続しないことがわかります。

## E. エージェントを学習させる


### 1. Tensorboardのセットアップ

学習の経過をモニタするため、Tensorboardをセットアップします。セルの出力に、インラインで表示することができます。

In [0]:
%load_ext tensorboard
%tensorboard --logdir cartpole_tensorboard

### 2. エージェントを学習させる

試しに1000ステップ学習を進めてみます。所要時間は1分程度です。


In [0]:
n_timesteps = 5000
model.learn(n_timesteps)

## F. 学習済みエージェントの評価

### 1. 学習済みエージェントの評価

学習後の平均の報酬(Mean reward)と、エピソード数(Num episodes)が得られます。学習前と比べてみましょう。報酬は大きく上昇し、エピソード数は減少しているのがわかるでしょうか。

In [0]:
# Evaluate the trained agent
mean_reward = evaluate(model, num_steps=10000)

### 2. 学習済みエージェントの振る舞いを見る

実際の振る舞いを再生してみましょう。うまくバランスを制御できていることが分かります。

In [0]:
frames = playback(model, env, 100)

plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames[0])
plt.axis('off')
animate = lambda i: patch.set_data(frames[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)
HTML(ani.to_jshtml())

### 3. 追加の学習を続け、振る舞いを見る

さらに、10,000ステップ追加で学習してみます。

In [0]:
n_timesteps = 10000
model.learn(n_timesteps)

In [0]:
frames = playback(model, env, 100)

plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames[0])
plt.axis('off')
animate = lambda i: patch.set_data(frames[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)
HTML(ani.to_jshtml())

さきほどと振る舞いは変わったのですが、どうも変な学習をしてしまったようです。

というのも、エピソードの開始とともに、片方向に移動し、その後もう片方向にずっと移動する、という同じスタイルが連続しています。
確かに、こうすることで一定時間プレイを続けて報酬を稼げるのですが、画面から出てしまうとそのエピソードが終了してしまうため、最適な状態は学習できていません。

# 発展

ここまで見てきたように、強化学習は、

- 環境を準備する
- エージェントを作成する
- 環境上で、エージェントに探索させる

というステップで学習を進めます。

RL Baselines Zooは、これらのステップを1行のコマンドの裏側にまとめてくれています。

## RL Baseline Zooのインストール

予め学習が収束するようハイパーパラメータなどが準備された1行のコマンドで、上記の学習や動画の生成を試してみましょう。また、DQNだけでなく、さまざまなエージェントのモデルを試すことができます。

まず、下のコマンドで、GitHubのリポジトリからrl-baselines-zooプロジェクトをコピーしてきます。

In [0]:
%cd /content
!git clone https://github.com/araffin/rl-baselines-zoo.git

次のコマンドで学習を進められます。実行すると、 `logs/dqn/CartPole-v1_1/` に  `CartPole-v1.pkl` として、学習済みエージェントのモデルが保存されます。

一般に、 `logs/{algorithm}/{env_name}_n` に、 `{env_name}.pkl` として保存されます。実行するごとに、n=1, 2, 3... と数が増加します。

また、RL Baselines Zooのリポジトリからは、 `/content/rl-baselines-zoo/trained_agents/` 以下に、各モデル・各環境の学習済みモデルが提供されています。

In [0]:
%cd /content/rl-baselines-zoo
!python train.py --algo dqn --env CartPole-v1 --tensorboard-log cartpole_tensorboard/

### 学習済みエージェントを読み込み、プレイバック

では、学習結果を見てみましょう。学習を複数回実行した場合は、`logs/dqn/CartPole-v1_n/CartPole-v1.zip` の、nの数を変えてください。

In [0]:
model = DQN.load("/content/rl-baselines-zoo/logs/dqn/CartPole-v1_1/CartPole-v1.zip")
env = gym.make("CartPole-v1")
env = DummyVecEnv([lambda: env]) 
model.set_env(env)

### プレイ動画の再生

学習結果を再生してみましょう。

In [0]:
frames = playback(model, env, 500)

plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames[0])
plt.axis('off')
animate = lambda i: patch.set_data(frames[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)
HTML(ani.to_jshtml())

## まとめと発展

このノートブックでは、CartPoleという基本のタスクを元に強化学習の流れを体感しました。

また、Stable Baselinesをそのまま使う場合と、RL Baselines Zooの1行コマンドで学習させる簡易な方法の2つを学びました。